1. Get Audio Information from Spotify
2. Insert into DB


In [33]:
import requests
from urllib.parse import urljoin

In [34]:
from pymongo import MongoClient
from pprint import pprint
from typing import List

# DB CONFIG
client = MongoClient('mongo', 27017)
db = client.music_db
COLLECTION = 'country-songs'
songs_collection = db[COLLECTION]

In [35]:
# Pull all songs from DB
SONGS = list(songs_collection.find())
len(SONGS)

9722

In [36]:
def has_lyrics(song):
    if not song.get('lyrics'):
        return False
    
    if song.get('lyrics') == 'null':
        return False

    if song.get('lyrics').get('error'):
        return False
        
    return True

songs = [song for song in SONGS]

In [37]:
class Song:
    def __init__(self, song: dict):
        self.data = song
        self.spotify = {}    # Should be assined with the spotify api
    
    @property
    def title(self):
        return self.data['title']
    
    @property
    def artist(self):
        return self.data['artist']
        
    @property
    def lyrics(self):
        return self.data['lyrics']['result']['track']['text']
    
    @property
    def search_phrase(self):
        return self.data['title'] + " by " + self.data['artist']
    
    def __str__(self):
        return str(self.data)
    
    def __repr__(self):
        return self.__str__()
    
song_list = list(map(Song, songs))

In [38]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

CLIENT_SECRET_KEY = 'c784fff30881479abc02c128d341efe2'
CLIENT_ID = '299217ad61af41beb3f025a8b9bf0d99'


client_credentials_manager = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET_KEY)
spotify = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [39]:
from typing import Iterable

def retrieve_track_id(name):
    results = spotify.search(q='track:' + name, type='track')
    return results

def chunk(it: Iterable, size: int):
    i = 0
    for j in range(size, len(it), size):
        yield it[i:j]
        i = j

In [52]:
from typing import List
from IPython.display import clear_output

    
def retrieve_and_assign_features(songs: List):
    song_ids = [song.spotify['id'] for song in songs]
    features = spotify.audio_features(tracks=song_ids)
    for song, feature in zip(songs, features):
        song.spotify['audio_features'] = feature
    return songs

def retrieve_and_assign_ids(song_list, start_index=0):
    i = 0
    for song in song_list[start_index:]:
        i += 1
        clear_output(wait=True)
        print(f"Retreiving ID: {i} for {song.search_phrase}\r")
        try:
            response = retrieve_track_id(song.search_phrase)
            song_id = response['tracks']['items'][0]['id']
            song.spotify['id'] = song_id
            yield song
        except IndexError:
            print("Couldn't retrieve ID for", song.search_phrase, response)
            continue
    
song_list = list(retrieve_and_assign_ids(song_list, 4748))

Retreiving ID: 3518 for That's When You Came Along by Tara Lyn Hart


SSLError: HTTPSConnectionPool(host='api.spotify.com', port=443): Max retries exceeded with url: /v1/search?q=track%3AThat%27s+When+You+Came+Along+by+Tara+Lyn+Hart&limit=10&offset=0&type=track (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')",),))

In [46]:
filtered_song_list = [song for song in song_list if song.spotify.get('id')]

In [47]:
i=0
for lst in chunk(filtered_song_list, 50):
    retrieve_and_assign_features(lst)
    print(i)
    clear_output(wait=True)
    i+=1

64


In [48]:
COLLECTION

'country-songs'

In [49]:
def update_db(document_id, spotify_data: dict):
    document = songs_collection.find_one_and_update({"_id": document_id}, 
                                         {"$set": {"spotify": spotify_data}})
    return document

def update_db_from_songs(songs):
    for song in songs:
        update_db(song.data['_id'], song.spotify)

update_db_from_songs(song_list)

In [50]:
# Get album IDs
SONGS = list(songs_collection.find())

In [51]:
songs = [song for song in SONGS if song.get('spotify', {}).get('id')]
len(songs)

3297